In [21]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

In [22]:
#Instalar SURPRISE en Anaconda Prompt
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [4]:
conn=sql.connect('db_movies')
cur=conn.cursor() 

Sistema de recomendación basado en contenido KNN con base en todo lo visto por el usuario

In [5]:
movies=pd.read_sql('select * from moviesready',conn)
movies['año_estreno']=movies.año_estreno.astype('int')

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2121 entries, 0 to 2120
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      2121 non-null   int64 
 1   title        2121 non-null   object
 2   cnt_rat      2121 non-null   int64 
 3   Action       2121 non-null   int64 
 4   Adventure    2121 non-null   int64 
 5   Animation    2121 non-null   int64 
 6   Children     2121 non-null   int64 
 7   Comedy       2121 non-null   int64 
 8   Crime        2121 non-null   int64 
 9   Documentary  2121 non-null   int64 
 10  Drama        2121 non-null   int64 
 11  Fantasy      2121 non-null   int64 
 12  Film-Noir    2121 non-null   int64 
 13  Horror       2121 non-null   int64 
 14  IMAX         2121 non-null   int64 
 15  Musical      2121 non-null   int64 
 16  Mystery      2121 non-null   int64 
 17  Romance      2121 non-null   int64 
 18  Sci-Fi       2121 non-null   int64 
 19  Thriller     2121 non-null 

In [6]:
##### cargar data frame escalado y con dummies ###

movies_dum1= joblib.load('salidas_movies_dum1.joblib')

usuarios=pd.read_sql('select distinct (user_id) as user_id from ratingsready',conn)

user_id=89

In [7]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ratings=pd.read_sql('select *from ratingsready where user_id=:user',conn, params={'user':user_id})
    l_movies_r=ratings['movieid'].to_numpy()
    
    ####agregar la columna de movie_id y title de la pelicula a dummie para filtrar y mostrar nombre
    movies_dum1[['movie_id','title']]=movies[['movieId','title']]

    ### filtrar peliculas calificadas por el usuario
    movies_r=movies_dum1[movies_dum1['movie_id'].isin(l_movies_r)]

    ## eliminar columna movie_id' y title
    movies_r=movies_r.drop(columns=['movie_id','title'])
    movies_r["indice"]=1 
        
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean() 

    ### filtrar peliculas no vistas    
    movies_nr=movies_dum1[~movies_dum1['movie_id'].isin(l_movies_r)] 
        
    ## eliminar movie_id y title
    movies_nr=movies_nr.drop(columns=['movie_id','title'])

    ### entrenar modelo 
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr) 
    dist, idlist = model.kneighbors(centroide)
        
    ids=idlist[0]
    recomend_b=movies.loc[ids][['title','movieId']]
    leidos=movies[movies['movieId'].isin(l_movies_r)][['title','movieId']]
    return recomend_b

recomendar(30)

print(interact(recomendar))


interactive(children=(Dropdown(description='user_id', options=(1, 410, 403, 404, 405, 406, 407, 408, 409, 411,…

<function recomendar at 0x0000027EB17859E0>


4. Sistema de recomendación filtro colaborativo.

In [13]:
#datos iniciales
ratings=pd.read_sql('select * from ratings_finalready where movie_rating>0', conn)
reader = Reader(rating_scale=(0, 10)) #escala calificacion
data   = Dataset.load_from_df(ratings[['user_id','movieid','movie_rating']], reader)

models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()] 
results = {}

###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación 
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación

#### for para probar varios modelos ##########
model=models[1]
for model in models:
 
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)  
    
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.642332,0.840164,0.269630,1.666633
knns.KNNWithZScore,0.647884,0.851174,0.239339,1.412487
knns.KNNWithMeans,0.653710,0.854171,0.239244,1.205872
knns.KNNBasic,0.694502,0.904134,0.211039,1.018601


In [15]:
###################se escoge el mejor knn withmeans#########################
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}
             }

## min support es la cantidad de items o usuarios que necesita para calcular recomendación
## name medidas de distancia
### se afina si es basado en usuario o basado en ítem

gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)
                                    
gridsearchKNNWithMeans.fit(data)


gridsearchKNNWithMeans.best_params["rmse"]
gridsearchKNNWithMeans.best_score["rmse"]
gs_model=gridsearchKNNWithMeans.best_estimator['rmse'] ### mejor estimador de gridsearch


In [20]:
################# Entrenar con todos los datos y Realizar predicciones con el modelo afinado

trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrnamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)



predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y las peliculas que no han calificado
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)


predictions = gs_model.test(predset) ### función muy pesada, hace las predicciones de rating para todas las peliculas que no ha visto un usuario
### la funcion test recibe un test set constriuido con build_test method, o el que genera crosvalidate

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings.
predictions_df.sort_values(by='est',ascending=False)

def recomendaciones(user_id,n_recomend=10):
    
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")
    
    recomendados=pd.read_sql('''select a.*, b.title 
                             from reco a left join movies_final b
                             on a.iid=b.movieid ''', conn)

    return(recomendados)


 
recomendaciones(user_id=500,n_recomend=10)

### ANALISIS:
###

Computing the msd similarity matrix...
Done computing similarity matrix.


,index,iid,est,title
0,996387,1041,4.415677,Secrets & Lies (1996)
1,996389,2360,4.403521,"Celebration, The (Festen) (1998)"
2,996456,1178,4.352519,Paths of Glory (1957)
3,995079,922,4.333181,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
4,996021,1104,4.321618,"Streetcar Named Desire, A (1951)"
5,996211,951,4.320151,His Girl Friday (1940)
6,994691,80906,4.309838,Inside Job (2010)
7,996421,905,4.276545,It Happened One Night (1934)
8,994723,162,4.262697,Crumb (1994)
9,995313,1221,4.252622,"Godfather: Part II, The (1974)"
